In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests

from bs4 import BeautifulSoup #library for web scrapping

import requests
import urllib.request
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

<h1>Part I
    <h2>This it the first part of the project that involves scrapping the wikipeadia page and manipulating the data in a pandas dataframe

In [41]:
#download the URL and extract the content to the variable html 

url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

request = urllib.request.Request(url)
html = urllib.request.urlopen(request).read()

soup = BeautifulSoup(html, "html.parser")

site_table = soup.table
column_names=[]

#fetching column names from the url
for data in site_table.find_all('th'):
    column_names.append(data.text.strip())
    

column_names

['Postcode', 'Borough', 'Neighbourhood']

In [42]:
#fetching the postal code information from the extracted data
info_data = []
for info in site_table.find_all('tr'):
    info_data.append(info.text.strip())

neighbourhood = []
list_tor = []

for i in range(len(info_data)):
    x = info_data[i].split('\n')
    list_tor.append(x)
    
#placing the extracted data in a pandas dataframe
for j in range(len(list_tor)-1):
    post = list_tor[j+1][0]
    bor = list_tor[j+1][1]
    nei = list_tor[j+1][2]
    neighbourhood.append({'Postcode': post, 'Borough': bor, 'Neighbourhood': nei})

neighbourhood = pd.DataFrame.from_dict(neighbourhood)
neighbourhood.rename(columns={'Postcode':'PostalCode'}, inplace=True)
neighbourhood.head()

,Borough,Neighbourhood,PostalCode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A


In [46]:
#re-arranging the columns
arranged_cols = [neighbourhood.columns[-1]] + [neighbourhood.columns[0]] + [neighbourhood.columns[1]]
df_toronto = neighbourhood[arranged_cols]
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [48]:
#dropping rows with not assigned boroughs
df_toronto.drop(df_toronto[df_toronto['Borough']=='Not assigned'].index,axis=0,inplace=True)
df_toronto.reset_index(inplace=True, drop=True)
df_toronto.head()

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [49]:
#replacing not assigned neighbourhood with its borough
df_toronto['Neighbourhood'] = df_toronto['Neighbourhood'].replace('Not assigned', df_toronto['Borough'])
df_toronto.head()

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor


In [50]:
#grouping neighbourhood with same postal code
df_grouped = df_toronto.groupby('PostalCode').agg(lambda x :', ' .join(set(x))).reset_index()
df_grouped.head(10)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Morningside, West Hill, Guildwood"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest"
9,M1N,Scarborough,"Cliffside West, Birch Cliff"


In [51]:
df_grouped.shape

(103, 3)

<h1> Part II
    <h2>Fetching the latitude and logitude of the postal codes

In [52]:
# The code was removed by Watson Studio for sharing.

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [53]:
#renaming the postal code column name so as to merge the coordinate df with the postal code df
df_data_0.rename(columns={'Postal Code': 'PostalCode'},inplace=True)

In [54]:
#merging the coordinates with the postal code, borough and neighbourhood
df_tor_ll = df_grouped.merge(df_data_0, on = 'PostalCode')

In [57]:
df_tor_ll

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside, West Hill, Guildwood",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview, East Birchmount Park, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Scarborough Village West, Cliffside, Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West, Birch Cliff",43.692657,-79.264848


<h1> Part III
    <h2>Clustering the Toronto Neighbourhood

In [58]:
#Filter for Toronto borough
Toronto = df_tor_ll[df_tor_ll.Borough.str.contains('Toronto')].reset_index(drop=True)
Toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"Riverdale, The Danforth West",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [62]:
#fetching the coordinates of Toronto
address = 'Toronto, ON'

geolocator = Nominatim(user_agent = 'matth_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The coordinates for Toronto are {}, {}'.format(latitude,longitude))

The coordinates for Toronto are 43.653963, -79.387207


In [60]:
# create map of Toronto using latitude and logitude
toronto_map = folium.Map(location=[43.653963, -79.387207], zoom_start=12)

#add markers to the map
for lat, lng, borough, neighborhood in zip(Toronto['Latitude'],Toronto['Longitude'], Toronto['Borough'], Toronto['Neighbourhood']):
    label = '{},{}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
            [lat,lng],
            radius=5,
            popup=label,
            color='green',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(toronto_map)

toronto_map

In [64]:
# The code was removed by Watson Studio for sharing.

In [65]:
radius = 500
limit = 30
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&oauth_token={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, 
                                                                                                                         CLIENT_SECRET,
                                                                                                                        ACCESS_TOKEN,
                                                                                                                        latitude,
                                                                                                                        longitude,
                                                                                                                        VERSION,
                                                                                                                        radius,
                                                                                                                        limit)

url


'https://api.foursquare.com/v2/venues/explore?client_id=ELJOKNNICDUJECAHB20M1K51H3QHCW4A3IDZ5LWG1VQLGZQN&client_secret=0AZC45PB3GDTRLQOGQ4GPKC2BC2LAJ3ETWVUXACXHHEYMTGB&oauth_token=OALZUQA3D1BSNEO1UNCSWYY5FCRD354Y0NEPJ5N3YA0XAYTC&ll=43.653963,-79.387207&v=20190202&radius=500&limit=30'

In [66]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4abff047e0d600269d8110'},
 'notifications': [{'type': 'notificationTray', 'item': {'unreadCount': 0}}],
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 110,
  'suggestedBounds': {'ne': {'lat': 43.6584630045, 'lng': -79.38099903084075},
   'sw': {'lat': 43.649462995499995, 'lng': -79.39341496915925}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labele